In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://results.eci.gov.in/PcResultGenJune2024/"

header = ['Party Name', 'Total Winning', 'Constituency', 'Winning Candidate', 'Total Votes', 'Margin', 'State']
main_data = []

response = requests.get(f"{base_url}index.htm")
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'lxml')

state_list = soup.find('select')

for option in state_list.find_all('option'):
    state_code = option.get('value')
    state_name = option.text.strip()

    if state_code:
        state_url = f"{base_url}partywiseresult-{state_code}.htm"
        state_response = requests.get(state_url)
        state_response.encoding = 'utf-8'
        state_soup = BeautifulSoup(state_response.text, 'lxml')


        state_table = state_soup.find("table")

        if state_table:
            for row in state_table.find_all('tr'):
                cols = [cell.text.strip() for cell in row.find_all(['td', 'th'])]
                if len(cols) >= 2 and cols[0] not in ["Total", "Party"]:
                    party_name, total_winning = cols[:2]


                    for link in state_soup.find_all('a', href=True):
                        if 'partywisewinresult' in link['href']:
                            constituency_url = base_url + link['href']
                            constituency_response = requests.get(constituency_url)
                            constituency_response.encoding = 'utf-8'
                            constituency_soup = BeautifulSoup(constituency_response.text, 'lxml')

                            # Loop through each row in the constituency table
                            for cons_row in constituency_soup.find_all('tr'):
                                cons_cols = [cell.text.strip() for cell in cons_row.find_all(['td'])]
                                if len(cons_cols) >= 5:
                                    constituency, winning_candidate, total_votes, margin = cons_cols[1:5]
                                    main_data.append([party_name, total_winning, constituency, winning_candidate, total_votes, margin, state_name])

df = pd.DataFrame(main_data, columns=header)
df.to_csv('election_data.csv', index=False)